In [4]:
#import xspharm
import numpy as np
import xarray as xr
import pandas as pd
#from doppyo import utils
from scipy import interpolate
import matplotlib.pyplot as plt
from cartopy import crs as ccrs
import matplotlib.ticker as mticker
#from doppyo import diagnostic as dgn
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

### Integrated water vapor transport (IVT)

Water vapor transport can be diagnosed on a global scale by vertically integrating water vapor flux in a column,

$$Q_\lambda= \frac{1}{g}\int_{sfc}^{300hPa}qu\ dp$$

$$Q_\phi= \frac{1}{g}\int_{sfc}^{300hPa}qv\ dp$$

where $Q_\lambda$ is the zonal water vapor transport, $Q_\phi$ the meridional water vapor transport, $g$ the gravimetric constant, $q$ specific humidity, $u$ and $v$ the zonal and meridional component of total flow, respectively. IVT is then the magnitude of water vapor transport,

$$IVT=\sqrt{ Q_\lambda ^{\,\,2}\ +\ Q_\phi ^{\,\,2} }$$

In [ ]:
#--Compute--#
qu = ugrd * spfh
qv = vgrd * spfh

qu_int = utils.integrate(qu, over_dim="level", x=(qu["level"] * 100) / 9.81)
qv_int = utils.integrate(qv, over_dim="level", x=(qv["level"] * 100) / 9.81)

IVT = np.sqrt(np.square(qu_int) + np.square(qv_int))

In [ ]:
#--Data written to disk--#
path = r'/home/bla390/control_run/data/control_6_no_convect.IVT.15000101_17991231.nc'
IVT = xr.open_dataset(path).IVT
IVT_seasonalMeans = IVT.groupby("time.season").mean("time")

In [ ]:
#== PLOT ==#

levels = [0,50,100,150,200,250,300,350,400,450,500]

fig = plt.figure(figsize=(10, 8))
gsFig = plt.GridSpec(1, 1)
gsFig.update(left=0.05, right=0.95, bottom = 0.10, top = 0.90, wspace=0.00, hspace = 0.00)
axFig = plt.subplot(gsFig[0, 0], projection=ccrs.PlateCarree(180))
image = IVT_seasonalMeans.sel(season="DJF").plot.contourf(ax=axFig, levels = levels, cmap=plt.cm.Blues, add_colorbar=False, 
                                   extend='both', transform=ccrs.PlateCarree())
axFig.coastlines()

#--Modify gridlines
gl = axFig.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
gl.xlabels_top = False ; gl.ylabels_right = False
gl.xformatter = LONGITUDE_FORMATTER ; gl.yformatter = LATITUDE_FORMATTER
gl.xlocator = mticker.FixedLocator([0, 60, 120, 180, -120, -60])
axFig.gridlines(crs=ccrs.PlateCarree(), draw_labels=False)
gl.xlabel_style = {'size': 12} ; gl.ylabel_style = {'size': 12}

#--Fix colorbar
cbar = plt.colorbar(image, ax=axFig, fraction=0.02, pad=0.03)

#--Labels
axFig.set_title('Integrated Water Vapor Transport - DJF',fontsize=18)

#--Save figure
#plt.savefig('/home/bla390/control_run/images/IVT_DJF.png')

### Rotational and divergent components of water vapor transport

Total water vapor transport, $\mathbf Q$, can be divided into its rotational and divergent components ($\mathbf Q_R$ and $\mathbf Q_D$, respectively) as follows,

$$ \mathbf Q= \mathbf Q_R + \mathbf Q_D = \hat{k} \times \nabla \psi_{\mathbf Q} + \nabla \chi_{\mathbf Q} $$

where $\psi_{\mathbf Q}$ is the streamfunction of water vapor transport, and $\chi_{\mathbf Q}$ the potential function of water vapor transport. Therefore, the divergent component of vertically integrated water vapor transport is equivalent to the gradient of water vapor transport potential function. This leads to the following relationship,

$$ \nabla^2 \chi_{\mathbf Q} = \nabla \cdot \mathbf Q $$

so that we may solve the Poisson equation for $\chi_{\mathbf Q}$ using our derived values of $Q_\lambda$ and $Q_\phi$. We may then numerically differentiate the potential function of water vapor transport to solve for the divergent component of water vapor transport,

$$ \mathbf Q_D = \frac{\partial \chi_{\mathbf Q}}{\partial x}\hat{i} + \frac{\partial \chi_{\mathbf Q}}{\partial y}\hat{j} $$